# Network Analysis on Auckland road data
### Import packages

In [8]:
import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd
from shapely.geometry import MultiPolygon, Point
from shapely.ops import transform
import pyproj
import networkx as nx
import math
import matplotlib.pyplot as plt
from pyproj import CRS
import itertools
import networkx as nx

### Importing the road network

In [ ]:
auck_network = ox.io.load_graphml("data/network_analysis/auckland_network_walk_consolidated.graphml")

In [ ]:
ox.plot_graph(auck_network)

# Adding other data
### Import the grid of urban Auckland including census variables
The IDW was done in R. Code in the file data-prep.R

In [ ]:
grid = gpd.read_file("data/geographic/grids/grid_50000_auckland_census_dist.gpkg")
grid.head()

### Import the train stations in Auckland
The IDW was done in R. Code in the file data-prep.R

In [ ]:
stations = gpd.read_file("data/transport/public_transport/trains_auckland_with_distances_on_network.gpkg")
stations.head()

## 


In [14]:
orig_xy = Point(174.762506, -36.852165)
target_xy = Point(174.762336, -36.852165)

orig_node = ox.get_nearest_node(auck_network, X=, Y=)
target_node = ox.get_nearest_node(auck_network, X=, Y=)

o_closest = nodes_proj.loc[orig_node]
t_closest = nodes_proj.loc[target_node]

orig_node
target_node

AttributeError: module 'osmnx' has no attribute 'get_nearest_node'

In [ ]:
route = nx.shortest_path(G=G, source=orig_node, target=target_node, weight='length')
print(route)